<h2>1.資料介紹</h2>

1.offering.txt 飯店基本資料:      
id:飯店ID(主key)  
name:飯店名稱  
hotel_class:飯店星等 (每0.5為單位，0~5星,<font color="red">這並非所有資料都有星等</font>)   
address: 包含街道、郵遞區號、城市等  
region_id: 所屬地區  
url: TripAdvisor 網址  
其他欄位如 details、phone 等 可能為空值  

2.review.txt 使用者評論資料:  
text: 評論內容  
ratings: 各項評分（overall, service, rooms, cleanliness, ...）  
title: 評論標題 
date, date_stayed: 留言與入住日期   
offering_id: forign key to 'id' in offering.txt   
author: 評論者資訊  


<h2>2.資料處理</h2>
2.1套件載入

In [12]:
# 套件載入
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt 
import re
import nltk
#nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math
from nltk import ngrams
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from collections import Counter, namedtuple

import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity


2.2原始資料json檔案轉為excel

In [13]:
# # offering.txt
# offering_df = pd.read_json("./raw_data/offering.txt", lines=True)
# offering_df.to_excel("./raw_data/offering.xlsx", index=False)
# print(" offering.xlsx done.")

# # review.txt read JSON and convert to Excel
# valid_reviews = []
# with open('./raw_data/review.txt', 'r', encoding='utf-8') as f:
#     for line in f:
#         try:
#             review = json.loads(line)
#             valid_reviews.append(review)
#         except json.JSONDecodeError:
#             continue  # ignore decode error

# # Convert to DataFrame and store in Excel.
# review_df = pd.DataFrame(valid_reviews)
# review_df.to_excel("./raw_data/review.xlsx", index=False)
# print(f"review.xlsc done, total {len(review_df)} rows.")


2.3left join合併兩份資料

In [14]:
# # join兩份資料
# merged_df = pd.merge(review_df, offering_df, left_on="offering_id", right_on="id", how="left")
# merged_df.to_excel("./raw_data/merged_reviews.xlsx", index=False)
# print(f" merged_reviews.xlsx's merged,total {len(merged_df)} rows.")

# # 拆分成有星等與沒星等
# has_class_df = merged_df[merged_df['hotel_class'].notna()]
# no_class_df = merged_df[merged_df['hotel_class'].isna()]

# # 儲存兩份 Excel 檔案
# has_class_df.to_excel("./raw_data/hotel_with_class.xlsx", index=False)
# no_class_df.to_excel("./raw_data/hotel_without_class.xlsx", index=False)

# # 列出各自筆數
# print(f" 有星等的飯店筆數：{len(has_class_df)}")
# print(f" 無星等的飯店筆數：{len(no_class_df)}")

In [15]:
reviews = pd.read_csv("./raw_data/reviews.csv")
offerings = pd.read_csv("./raw_data/offerings.csv")
offerings = offerings[offerings['hotel_class'].notna()]
has_class_df = pd.merge(reviews, offerings, left_on="offering_id", right_on="id", how="left")
print(has_class_df.head(2))

                                             ratings  \
0  {'service': 5.0, 'cleanliness': 5.0, 'overall'...   
1  {'service': 5.0, 'cleanliness': 5.0, 'overall'...   

                                       title  \
0  “Truly is "Jewel of the Upper Wets Side"”   
1                  “My home away from home!”   

                                                text  \
0  Stayed in a king suite for 11 nights and yes i...   
1  On every visit to NYC, the Hotel Beacon is the...   

                                              author    date_stayed  \
0  {'username': 'Papa_Panda', 'num_cities': 22, '...  December 2012   
1  {'username': 'Maureen V', 'num_reviews': 2, 'n...  December 2012   

   offering_id  num_helpful_votes        date       id_x  via_mobile  \
0        93338                  0  2012-12-17  147643103       False   
1        93338                  0  2012-12-17  147639004       False   

   hotel_class  region_id                                                url  \
0     

2.5停用字

In [16]:
##新增stop words
# nltk.download('stopwords')
# stopWords = stopwords.words('english')

2.6斷詞

In [17]:
# 設定斷詞 function
# stemmer = PorterStemmer() #做Stemming還原詞幹

# def getToken(row):
#     seg_list = nltk.tokenize.word_tokenize(row)
#     seg_list = [stemmer.stem(str(w)).lower() for w in seg_list if w not in stopWords and len(w)>1]
#     seg_list = [w for w in seg_list if w not in stopWords]
#     return seg_list

In [18]:
# 停用、斷詞、詞幹處理
nltk.download('punkt')
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))
custom_stopwords = stopWords.union({"''", "'", "`", "``", ".", ",", "...", "!", "？", "。", ":", ";", "-", "--", "(", ")", "…"})
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 建立 data['word'] 欄位（保留原始詞彙list）
data = has_class_df.copy()
data["text"] = data["text"].fillna("").astype(str)
def getToken(text):
    tokens = word_tokenize(text)
    tokens = [stemmer.stem(w.lower()) for w in tokens if w.lower() not in custom_stopwords and len(w) > 1]
    return tokens

data = has_class_df.copy()
data["text"] = data["text"].fillna("").astype(str)  # 將 NaN → 空字串，並確保是字串
data['word'] = data['text'].apply(getToken)
# data = data.explode('word')
# data.reset_index(inplace=True, drop=True) #重設index
# data.head()

In [ ]:
# 轉為 (offering_id, word) 組合以統計詞頻
pair_list = []
for offering_id, word_list in zip(data['offering_id'], data['word']):
    for w in word_list:
        pair_list.append((offering_id, w))

# 建立 DataFrame
word_data = pd.DataFrame(pair_list, columns=['offering_id', 'word'])

<h2>3.TF-IDF</h2>
直接使用公式計算

In [ ]:
# 各hotel的總詞彙數
# total_words = data.groupby(['offering_id'],as_index=False).size()
# total_words.rename(columns={'size': 'total'}, inplace=True)
# total_words

In [ ]:
# 計算各詞彙在各飯店出現的次數
# word_count = data.groupby(["offering_id","word"],as_index=False).size()
# word_count.rename(columns={'size': 'count'}, inplace=True)
# word_count

In [ ]:
# 各 hotel 的總詞彙數
total_words = word_data.groupby(['offering_id'], as_index=False).size()
total_words.rename(columns={'size': 'total'}, inplace=True)
total_words
# 各詞彙在各飯店出現的次數
word_count = word_data.groupby(["offering_id", "word"], as_index=False).size()
word_count.rename(columns={'size': 'count'}, inplace=True)
word_count

#發現"雙引號"與"單引號"被多次斷詞，因此手動加入停用字並移除

In [ ]:
# 取出英文停用詞清單
en_stopwords = stopwords.words("english")
# 建立 set 結構方便後續過濾
custom_stopwords = set(en_stopwords)
# 加入自訂的雜訊符號
custom_stopwords.update(["''", "'", "`", "``", ".", ",", "...", "!", "？", "。", ":", ";", "-", "--", "(", ")", "…" ])

data = data[~data["word"].isin(custom_stopwords)]
# 重新計算各詞彙在各飯店出現的次數
word_count = data.groupby(["offering_id","word"],as_index=False).size()
word_count.rename(columns={'size': 'count'}, inplace=True)
word_count


#### 合併需要的資料欄位
- 合併 **每個詞彙在每個飯店中出現的次數** 與 **每個飯店的總詞數**

In [ ]:
tp_words = word_count.merge(total_words,on = 'offering_id',how = 'left')

tp_words.head(10)

#### 以每個"飯店"爲單位，計算每個詞彙的 tf-idf 值  
- tf-idf = tf * idf

In [ ]:
# 計算tf
tp_words_tf_idf = tp_words.assign(tf = tp_words.iloc[:,2]/tp_words.iloc[:,3])
tp_words_tf_idf.head()

In [ ]:
# 計算每個詞彙出現在幾間評論中
idf_df = tp_words.groupby(['word'],as_index=False).size()
tp_words_tf_idf = tp_words_tf_idf.merge(idf_df,on = 'word',how = 'left')
tp_words_tf_idf.head()

In [ ]:
# 計算idf
tp_words_tf_idf = tp_words_tf_idf.assign(idf = tp_words_tf_idf.iloc[:,5]
                                               .apply(lambda x: math.log((len(total_words)/x),10)))

tp_words_tf_idf = tp_words_tf_idf.drop(labels=['size'],axis=1)
tp_words_tf_idf.head()

In [ ]:
# 計算 tf-idf
tp_words_tf_idf = tp_words_tf_idf.assign(tf_idf = tp_words_tf_idf.iloc[:,4] * tp_words_tf_idf.iloc[:,5])
tp_words_tf_idf.head(10)

 <h2>使用套件</h2>

In [ ]:
tp_df = has_class_df.copy()
# 需要改成使用空格連接斷好的詞
tp_df['text'] = tp_df['text'].fillna("").astype(str)
tp_df['word'] = tp_df.text.apply(getToken).map(' '.join)
tp_df.head()

In [ ]:
# Bag of Word 
tp_df_subset = tp_df.head(30000)

# 建立 CountVectorizer 並轉換
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tp_df_subset["word"])
vocabulary = vectorizer.get_feature_names_out()

# 建立 DataFrame（轉 dense 可能仍會吃記憶體）
DTM_df = pd.DataFrame(data=X.toarray(), columns=vocabulary)

# 顯示前幾筆
DTM_df.head()

In [ ]:
transformer = TfidfTransformer()  
# 將詞頻矩陣X統計成TF-IDF值
tfidf = transformer.fit_transform(X)  

# 轉成dataframe
TFIDF_df = pd.DataFrame(columns = vocabulary, data = tfidf.toarray())

TFIDF_df

### 3.2 檢視結果
根據tfidf值來挑出重要詞彙的方法有許多種，以下舉兩個方法當範例：
- 取每一個字詞在所有文件裡的TF-IDF平均值
- 取每份文件裡TF-IDF值最大的前10個字詞當為常用字詞，再取每一個字詞在常用字詞中出現頻率

#### 方法1： 取每一個字詞在所有文件裡的TF-IDF平均值

In [ ]:
tp_tfidf = TFIDF_df.mean().to_frame().reset_index()
tp_tfidf.columns = ["word", "avg"]

tp_tfidf.sort_values('avg', ascending = False).head(10)

#### 方法2： 取每份文件裡TF-IDF值最大的前10個字詞當為常用字詞，再取每一個字詞在常用字詞中出現頻率

In [ ]:
toptens = TFIDF_df.copy()
toptens.insert(0, 'offering_id', toptens.index+1)

toptens

In [ ]:
toptens = toptens.melt(id_vars = "offering_id", var_name = "word", value_name = 'tfidf')
toptens

nlargest() 會回傳指定列中最大的前x個值所對應的資料

In [ ]:
# 只取前 30000 筆（依資料量可再調整）
toptens_subset = toptens.head(100000)

top_words = (
    toptens_subset.groupby("offering_id")
    .apply(lambda x: x.nlargest(10, "tfidf"))
    .reset_index(drop=True)
    .groupby("word", as_index=False)
    .size()
    .sort_values("size", ascending=False)
    .head(10)
)

top_words

<h2>4.斷詞與N-gram</h2>
N-gram 指文本中連續出現的n個語詞。 透過N-gram我們可以找出有哪些詞彙較常一起出現，檢查是否需要加入自定義字典中。  
N-gram 範例：

4.1 Bi-gram

In [ ]:
# 設定 bigram 斷詞 function
def bigram_getToken(row):
    seg_list = nltk.tokenize.word_tokenize(row)
    seg_list = [stemmer.stem(str(w)).lower() for w in seg_list if w not in stopWords and len(w)>1]
    seg_list = [w for w in seg_list if w not in stopWords]
    seg_list = ngrams(seg_list, 2)
    seg_list = [" ".join(w) for w in list(seg_list)]
    return seg_list

In [ ]:
tp_bigram = has_class_df.copy()

# 先處理非字串的 text 欄位
tp_bigram["text"] = tp_bigram["text"].fillna("").astype(str)

tp_bigram["word"] = tp_bigram.text.apply(bigram_getToken)
tp_bigram = tp_bigram.explode('word')
tp_bigram

#### 統計最常出現的bigram組合

In [ ]:
# 計算每個組合出現的次數
tp_bigram_count = tp_bigram.groupby(["word"],as_index=False).size()
tp_bigram_count.sort_values(by=['size'], ascending=False).head(15)

4.2 Tri-gram

In [ ]:
# 設定 trigram 斷詞 function
def trigram_getToken(row):
    seg_list = nltk.tokenize.word_tokenize(row)
    seg_list = [stemmer.stem(str(w)).lower() for w in seg_list if w not in stopWords and len(w)>1]
    seg_list = [w for w in seg_list if w not in stopWords]
    seg_list = ngrams(seg_list, 3)
    seg_list = [" ".join(w) for w in list(seg_list)]
    return seg_list

In [ ]:
tp_trigram = has_class_df.copy()
tp_trigram["text"] = tp_trigram["text"].fillna("").astype(str)
tp_trigram["word"] = tp_trigram.text.apply(trigram_getToken)
tp_trigram = tp_trigram.explode('word')
tp_trigram

#### 統計最常出現的trigram組合

In [ ]:
# 計算每個組合出現的次數
tp_trigram_count = tp_trigram.groupby(["word"],as_index=False).size()
tp_trigram_count.sort_values(by=['size'], ascending=False).head(15)

<h2>5.Bi-gram視覺化</h2>

In [ ]:
#根據剛剛的觀察可以增加stopwords
stopWords.extend(['would', 'could', 'should', 'may', 'might', 'though'])

In [ ]:
# 使用FreqDist 取得 bigram 斷詞 與 bigram出現頻率
bigramfdist = FreqDist()
def bigram_getToken_dict(row):
    seg_list = nltk.tokenize.word_tokenize(row)
    seg_list = [stemmer.stem(str(w)).lower() for w in seg_list if w not in stopWords and len(w)>1]
    seg_list = [w for w in seg_list if w not in stopWords]
    bigramfdist.update(ngrams(seg_list, 2))
    seg_list2 = ngrams(seg_list, 2)
    seg_list = [" ".join(w) for w in list(seg_list2)]
    return seg_list

In [ ]:
#記憶體不足可能會crash
# 取 300,000 筆資料，視硬體規格調整
fud_bigram2 = has_class_df.head(300000).copy()
fud_bigram2["text"] = fud_bigram2["text"].fillna("").astype(str)
fud_bigram2["word"] = fud_bigram2["text"].apply(bigram_getToken_dict)
# 展開成一詞一列
fud_bigram2 = fud_bigram2.explode('word')
# 顯示前幾筆結果
fud_bigram2.head(3)


In [ ]:
# 看一下bigramfdist內容
bigramfdist.most_common(5)

In [ ]:
# 建立bigram和count的dictionary
# 這裡取最多的前30項
d = {k:v for k,v in bigramfdist.most_common(30)}
d

In [ ]:
# Create network plot 
G = nx.Graph()

# 建立 nodes 間的連結
for k, v in [d][0].items():
    G.add_edge(k[0], k[1], weight=v)
    
# 取得edge權重
weights = [w[2]['weight']*0.05 for w in  G.edges(data=True)]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

pos = nx.spring_layout(G, k=2)

# networks
nx.draw_networkx(G, pos,
                 font_size=16,
                 width=weights,
                 edge_color='grey',
                 node_color='purple',
                 with_labels = False,
                 ax=ax)

# 增加 labels
for key, value in pos.items():
    x, y = value[0]+.07, value[1]+.045
    ax.text(x, y,
            s=key,
            bbox=dict(facecolor='red', alpha=0.25),
            horizontalalignment='center', fontsize=13)
    
plt.show()

## 6. Pairwise correlation
6.1計算兩個詞彙間的相關性 Pearson correlation

In [ ]:
data_cor = has_class_df.copy()

# 需要改成使用空格連接斷好的詞
data_cor["text"] = data_cor["text"].fillna("").astype(str)
data_cor['word'] = data_cor.text.apply(getToken).map(' '.join)
data_cor

In [ ]:
# 只取前 10000 筆資料（記憶體crash)
data_subset = data_cor.head(10000)

# Bag of Word：限制出現在至少 5 篇文章中，且僅取前 300 個高頻詞
vectorizer = CountVectorizer(min_df=5, max_features=300)
X = vectorizer.fit_transform(data_subset["word"])
vocabulary = vectorizer.get_feature_names_out()

# 轉成 dataframe
DTM_df = pd.DataFrame(columns=vocabulary, data=X.toarray())
DTM_df.head()


In [ ]:
# 計算詞之間的相關係數
corr_matrix = np.corrcoef(DTM_df.T)

# 轉成dataframe
Cor_df = pd.DataFrame(corr_matrix, index = DTM_df.columns, columns = DTM_df.columns)

Cor_df.insert(0, 'word1', Cor_df.columns)
Cor_df.reset_index(inplace = True, drop = True)

Cor_df

In [ ]:
word_cor_df = Cor_df.melt(id_vars = 'word1', var_name = 'word2', value_name = 'cor')

# 去除兩個詞相同的情況
word_cor_df = word_cor_df[word_cor_df["word1"] != word_cor_df["word2"]]

word_cor_df.sort_values('cor', ascending=False).head(10)

### 6.2 和 clean, service 相關性最高的 8 個詞彙

In [ ]:
d_sum = word_cor_df[(word_cor_df["word1"]=="clean")].sort_values(by=['cor'], ascending = False).head(8)
m_sum = word_cor_df[(word_cor_df["word1"]=="service")].sort_values(by=['cor'], ascending = False).head(8)

In [ ]:
d_sum = d_sum.sort_values(by=['cor'])
m_sum = m_sum.sort_values(by=['cor'])

plt.figure(figsize=(12,8))   # 顯示圖框架大小 (寬,高)
plt.style.use("ggplot")     # 使用ggplot主題樣式
# plt.rcParams['font.sans-serif']=['SimHei'] #使中文能正常顯示

plt.subplot(121)
plt.title('mr_darcy')
plt.xlabel('cor')
plt.barh(d_sum['word2'],d_sum['cor'])

plt.subplot(122)
plt.title('marry')
plt.xlabel('cor')
plt.barh(m_sum['word2'],m_sum['cor'],color="darkblue")


plt.show()

## 7. 計算章節相似度
以TF-IDF的結果當作章節的向量，計算 Cosine Similarity 找出相似的章節   

In [ ]:
data_cos = data_cor.copy()
data_cos.head()

In [ ]:
#crash
# 只取前 10000 筆資料（記憶體crash)
data_subset2 = data_cor.head(10000)
# Bag of Word
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_subset2["word"])
vocabulary = vectorizer.get_feature_names_out()
# 轉成dataframe
DTM_df = pd.DataFrame(columns = vocabulary, data = X.toarray())
DTM_df.head()

In [ ]:
transformer = TfidfTransformer()  
# print(transformer) 
# 將詞頻矩陣X統計成TF-IDF值
tfidf = transformer.fit_transform(X)  
# print(tfidf.toarray())

# 轉成dataframe
TFIDF_df = pd.DataFrame(columns = vocabulary, data = tfidf.toarray())
TFIDF_df.head()

計算飯店間的cosine similarity

In [ ]:
cosine_matrix = cosine_similarity(tfidf.toarray(), tfidf.toarray())

找和"飯店1"星等相似的其他飯店

In [ ]:
cos_df = pd.DataFrame(cosine_matrix[1], columns = ['cos_similarity'])
cos_df

In [ ]:
cos_df = cos_df.merge(data_cos, how = 'left', left_index=True, right_index=True)
cos_df.loc[:,["cos_similarity","chapter","sentence"]].sort_values(by=['cos_similarity'], ascending=False).head(10)